<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## mmlupro_philosophy ETHICS dataset

In [1]:
# set parameters
 
file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()
file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [2]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")
# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

## 1. build DB (using Wikipida)

### what is the main topic & specific topic of query

In [3]:
# read mmlu_pro.csv file
import pandas as pd
import os

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

prompts, answers = read_data(os.path.join(data_path, 'mmlupro_test_philosophy.csv'))

In [4]:
testdata = pd.read_csv(data_path+'mmlupro_test_philosophy.csv')
testdata

,prompts,answers
0,QUESTION0) Select the best translation into pr...,A
1,QUESTION1) Construct a complete truth table f...,D
2,QUESTION2) Which of the following proposition...,I
3,QUESTION3) Select the best translation into pr...,A
4,QUESTION4) Select the best translation into pr...,E
...,...,...
494,QUESTION494) Mill claims that in deciding how ...,C
495,QUESTION495) Of the two versions of the princi...,A
496,"QUESTION496) According to Kant, laws of nature...",C
497,QUESTION497) Hare refers to people who endorse...,A


In [5]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers', 'top1', 'top2', 'top3'])

for index, row in testdata.iterrows():
    #if index % 10 != 0 : continue # 일단 실험할 땐 개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]

    try : 
        embedded_query = query_embeddings.embed_query(question) # 질문만 받아와서 embedding 하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a, 'top1': None, 'top2': None, 'top3': None}
    except :
        print(f'pass: {index}')
        continue


In [52]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3
0,0,"[-0.00475311279296875, 0.00991058349609375, -0...",Select the best translation into predicate log...,Select the best translation into predicate log...,A,None,None,None
1,1,"[0.0004830360412597656, -0.0250091552734375, -...",Construct a complete truth table for the foll...,Construct a complete truth table for the foll...,D,None,None,None
2,2,"[0.0006518363952636719, 0.004669189453125, -0....",Which of the following propositions is an imm...,Which of the following propositions is an imm...,I,None,None,None
3,3,"[-0.002498626708984375, -0.007579803466796875,...",Select the best translation into predicate log...,Select the best translation into predicate log...,A,None,None,None
4,4,"[-0.0167694091796875, -0.007282257080078125, -...",Select the best translation into predicate log...,Select the best translation into predicate log...,E,None,None,None
...,...,...,...,...,...,...,...,...
494,494,"[-0.0107269287109375, 0.0018281936645507812, -...",Mill claims that in deciding how to act:\n,Mill claims that in deciding how to act:\n(A) ...,C,None,None,None
495,495,"[-0.00873565673828125, -0.0032100677490234375,...",Of the two versions of the principle that Sing...,Of the two versions of the principle that Sing...,A,None,None,None
496,496,"[-0.0109405517578125, -0.0105133056640625, -0....","According to Kant, laws of nature are laws acc...","According to Kant, laws of nature are laws acc...",C,None,None,None
497,497,"[-0.0151214599609375, -0.0167999267578125, -0....",Hare refers to people who endorse ideals witho...,Hare refers to people who endorse ideals witho...,A,None,None,None


In [7]:
nowtest.question[1]

' Construct a complete truth table for the following pairs of propositions. Then, using the truth tables, determine whether the statements are logically equivalent or contradictory. If neither, determine whether they are consistent or inconsistent. Justify your answers.\n(~M ⊃ ~N) ∨ (O ≡ N) and (~M · N) · [(~O ∨ ~N) · (O ∨ N)]\n'

## db

In [10]:
import numpy as np
# .npy 파일 로드 (타입==넘파이)
ethicsDB_embed = np.load(data_path+'embedding/embedded100_ETHICSdataset.npy')
ethicsDB_embed = ethicsDB_embed.tolist()

ethicsDB = np.load(data_path+'embedding/chunked100_ETHICSdataset.npy')
ethicsDB = ethicsDB.tolist()

In [12]:
ethicsDB

["Aren't you supposed to be preparing for the meeting? No because I have everything already set up in the conference room. Could you take the car to the carwash today? But the car is already clean. Aren't you supposed to pay you sister back for the video game she bought you? No because it was my brother who bought for me. Aren't you supposed to remove mildew from the grout. No because removing that might dislodge the grout itself, it's very old.  Aren't you supposed to remove mildew from the grout. No because I have a skin sensitivity to the grout cleaner. Aren't you supposed to get the tickets for the fishing tour? No because I paid for them already. Aren't you supposed to get the tickets for the fishing tour? No because they were already booked. Could you write a letter to your mother.  But my mother died twenty years ago.  Aren't you supposed to be mowing the lawn? No because I'm out of gas. Aren't you supposed to be mowing the lawn? No because the lawn mower needs repair. Could you

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Please provide most correct answer. Let's think step by step.
    
    When translating the answer, do not exlain anything. And you must also include the choice number like :
    Answer : (Number) ~
    ---
    
    Question: {question}
    Context: {context}
    Answer :
    ---
        
    '''

)
zs_chain = prompt_template | llm


In [14]:
import numpy as np

responses = []
for idx, row in nowtest.iterrows() : # 질문 받아오기 
    if idx == 20 : break

    embed_ques= row.embed_ques

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embed_ques) @ np.array(ethicsDB_embed).T).argsort()[::-1]

    nowtest.loc[idx, 'top1'] = ethicsDB[sorted_idx[0]]
    nowtest.loc[idx, 'top2'] = ethicsDB[sorted_idx[1]]
    nowtest.loc[idx, 'top3'] = ethicsDB[sorted_idx[2]]

    response = zs_chain.invoke({"question": row.prompts, "context": ethicsDB[sorted_idx[0]]}) # top1에 대해서는 바로 답변 출력
    responses.append(response.content)

    nowtest.loc[idx, 'predict'] = response.content

responses

['Answer: (A) Tdc',
 'Answer: (E) Neither logically equivalent nor contradictory, but inconsistent',
 '(F) ~(U ⊃ W)',
 'Answer: (A) Lhf',
 'Answer: (D) ~eBj',
 'Answer: (I) (∀x)[Bx ⊃ (∃y)(Ny • Lxy)]',
 '(B) Invalid. Counterexample when L, M, O, Q, and R are true and N and P are false',
 '(J) It is wrong for society to kill a murderer.',
 'Answer: (A) we should refrain from imposing the death penalty.',
 'Answer: (F) the difficult task of prioritizing multiple global issues.',
 'Answer: (C)',
 '(E)',
 'Answer : (F) creating a global consensus on acceptable genetic engineering practices',
 'Answer: (B)',
 'Answer: (B)',
 '(I) people are allowed to do a smaller evil to prevent a greater one.',
 'Answer: (C) ~',
 'Answer: (A) ~',
 'Answer : (D)',
 'Answer: (C) successful interrogation may not require torture.']

## Check Accuracy

In [15]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [16]:
try : del [[wrongdf]]
except : pass
wrongdf = pd.DataFrame(columns=['questionNum', 'answer', '1st_pred'])

In [17]:
######### 정답 1차 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, responses)):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":"-"}
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":None}
        
acc = cnt/20*100
print(f"acc: {acc}%")
print()
print("1st wrong:", wrong)
wrongdf.loc[len(wrongdf)] = {"questionNum":"-", "answer" : "-", "1st_pred":acc}

----------
Answer: (A) Tdc
idx: 0 | generated answer: A, answer: A
----------
Answer: (E) Neither logically equivalent nor contradictory, but inconsistent
idx: 1 | generated answer: E, answer: D
----------
(F) ~(U ⊃ W)
idx: 2 | generated answer: F, answer: I
----------
Answer: (A) Lhf
idx: 3 | generated answer: A, answer: A
----------
Answer: (D) ~eBj
idx: 4 | generated answer: D, answer: E
----------
Answer: (I) (∀x)[Bx ⊃ (∃y)(Ny • Lxy)]
idx: 5 | generated answer: I, answer: I
----------
(B) Invalid. Counterexample when L, M, O, Q, and R are true and N and P are false
idx: 6 | generated answer: B, answer: D
----------
(J) It is wrong for society to kill a murderer.
idx: 7 | generated answer: J, answer: J
----------
Answer: (A) we should refrain from imposing the death penalty.
idx: 8 | generated answer: A, answer: D
----------
Answer: (F) the difficult task of prioritizing multiple global issues.
idx: 9 | generated answer: F, answer: H
----------
Answer: (C)
idx: 10 | generated answer

In [18]:
wrongdf

,questionNum,answer,1st_pred
0,2,D,E
1,3,I,F
2,5,E,D
3,7,D,B
4,9,D,A
5,10,H,F
6,11,D,C
7,12,G,E
8,13,B,F
9,14,D,B
